#### COCO JSON 파일을 사용하여 학습 시점에 즉석에서 마스크를 생성

별도의 마스크 이미지 파일 없이 원본 이미지와 JSON 파일만 있으면 동작하며, 
Lanemark와 같은 중요 클래스에 가중치를 두어 성능을 높이도록 설계됨. 

사전 준비
```
$ pip install pycocotools
```

In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, models
from pycocotools.coco import COCO
from PIL import Image
import numpy as np

1. COCO JSON 기반의 커스텀 데이터셋 정의
* 데이터를 로드, 전처리
	* cat_id_to_index 매핑: 
		* COCO JSON의 카테고리 ID는 보통 불연속적(예: 1, 15, 23)입니다. 
		* 이를 신경망 학습에 적합하도록 0, 1, 2... 형태의 연속적인 인덱스로 변환합니다.
	* On-the-fly 마스크 생성: 
		* coco.annToMask(ann) 함수를 사용하여 JSON의 폴리곤 좌표 데이터를 픽셀 형태의 이진 마스크로 바꾼 뒤, 해당 클래스 번호를 할당합니다.
	* 보간법(Interpolation) 처리:
		* 이미지 크기를 조절할 때는 일반적인 보간법을 쓰지만, 마스크(정답)의 경우 값이 뭉개지면 안 되므로 반드시 NEAREST (최근접 이웃) 보간법을 사용합니다。

In [2]:
class MotorcycleCocoDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_file, transform=None, num_classes=6):
        self.img_dir = img_dir
        self.coco = COCO(ann_file) # JSON 파일 로드
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transform = transform
        self.num_classes = num_classes

        # 카테고리 id들을 0..N-1 인덱스로 매핑
        self.cat_ids = sorted(self.coco.getCatIds())
        cats = self.coco.loadCats(self.cat_ids)
        # map original category_id -> continuous index (0..)
        self.cat_id_to_index = {cat['id']: idx for idx, cat in enumerate(cats)}
        if len(self.cat_id_to_index) > self.num_classes:
            raise ValueError(f"Found {len(self.cat_id_to_index)} categories but num_classes={self.num_classes}")

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        
        # 이미지 로드
        path = coco.loadImgs(img_id)[0]['file_name']
        # 이미지 로드 (안전하게 리스트 인자로 전달)
        img_info = coco.loadImgs([img_id])[0]
        try:
            img = Image.open(os.path.join(self.img_dir, img_info['file_name'])).convert('RGB')
        except Exception as e:
            raise RuntimeError(f"Failed to load image {img_info.get('file_name')} : {e}")

        # JSON의 폴리곤 좌표를 사용하여 픽셀 마스크 생성
        mask = np.zeros((img.height, img.width), dtype=np.uint8)
        for ann in anns:
            kind_mask = coco.annToMask(ann).astype(bool)
            cat_id = ann.get('category_id')
            mapped = self.cat_id_to_index.get(cat_id)
            if mapped is None:
                # unknown category -> skip (treated as background)
                continue
            mask[kind_mask] = mapped

        mask = Image.fromarray(mask)

        if self.transform:
            img = self.transform(img)
            # 마스크는 보간법 없이 크기만 조정
            mask = transforms.Resize((256, 256), interpolation=Image.NEAREST)(mask)
            mask = torch.from_numpy(np.array(mask)).long()
        else:
            mask = torch.from_numpy(np.array(mask)).long()

        # 안전 검사: 레이블 값이 허용 범위 내인지 확인
        if mask.max() >= self.num_classes:
            raise ValueError(f"Mask label {int(mask.max())} >= num_classes ({self.num_classes})")

        return img, mask

    def __len__(self):
        return len(self.ids)

2. get_criterion 함수 (가중치 기반 손실 함수)
  * 세그멘테이션 데이터셋의 고질적인 문제인 클래스 불균형을 해결하기 위한 전략
  * 클래스별 가중치: 배경(1.0)이나 도로(1.0)에 비해 픽셀 면적이 좁고 중요한 차선(Lanemark, 8.0)과 이동 물체(Movable, 4.0)에 높은 가중치를 부여한다. 
  * 이렇게 하면 모델이 차선처럼 작은 영역을 틀렸을 때 훨씬 큰 벌칙(Loss)을 받아 해당 부분을 더 집중해서 학습한다.

In [6]:
def get_criterion(device):
    # 야간 주행 시 식별이 어려운 Lanemark(차선) 등에 높은 가중치 부여 [cite: 17, 23]
    # 순서: [Background/Undrivable, Lanemark, Road, Movable, My bike, Rider]
    weights = torch.tensor([1.0, 8.0, 1.0, 4.0, 2.0, 2.0], dtype=torch.float).to(device)
    return nn.CrossEntropyLoss(weight=weights)

3. 메인 학습 루프
	* DeepLabV3 ResNet50: 구글이 개발한 고성능 세그멘테이션 모델
	* Optimizer: 보편적으로 성능이 좋은 Adam을 선택했습니다.
	* 데이터 흐름: Batch Size 4로 이미지를 입력받아 6개 클래스에 대한 예측 맵을 생성하고, 실제 마스크와 비교하여 가중치가 적용된 CrossEntropyLoss를 계산합니다.

In [4]:
def main():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # 재현성 설정
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    
    # 데이터 설정 (이미지 200프레임 기반) [cite: 8]
    data_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # 경로 설정 (실제 경로에 맞게 수정)
    dataset = MotorcycleCocoDataset(
        img_dir='./data/images', 
        ann_file='./data/annotations.json', 
        transform=data_transform
    )
    
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
    
    # 모델: DeepLabV3 ResNet50 (6개 클래스 분류용)
    # torchvision API differs by version; try common signatures
    try:
        model = models.segmentation.deeplabv3_resnet50(weights=None, num_classes=6).to(device)
    except TypeError:
        model = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=6).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = get_criterion(device)

    print("Start Trainning ...")
    for epoch in range(10):
        model.train()
        epoch_loss = 0
        for imgs, masks in dataloader:
            imgs, masks = imgs.to(device), masks.to(device)
            
            outputs = model(imgs)['out']
            loss = criterion(outputs, masks)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            
        print(f"Epoch {epoch+1}: Loss = {epoch_loss/len(dataloader):.4f}")

In [5]:
#if __name__ == "__main__":
main()

loading annotations into memory...
Done (t=2.01s)
creating index...
index created!
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 138MB/s] 


Start Trainning ...
Epoch 1: Loss = 0.6496
Epoch 2: Loss = 0.2991
Epoch 3: Loss = 0.2238
Epoch 4: Loss = 0.1894
Epoch 5: Loss = 0.1695
Epoch 6: Loss = 0.1515
Epoch 7: Loss = 0.1369
Epoch 8: Loss = 0.1255
Epoch 9: Loss = 0.1212
Epoch 10: Loss = 0.1132
